In [1]:
import numpy as np
import tonic
import tonic.transforms as transforms
import pandas as pd
from torch.utils.data import Dataset, DataLoader
#from tonic.dataset import Dataset
from typing import Callable, Optional
import torch
import matplotlib.pyplot as plt
from torchvision.ops import masks_to_boxes
from torchvision.utils import draw_bounding_boxes
import matplotlib.patches as patches
from tqdm.notebook import tqdm

In [2]:
import sys
sys.path.append("./mnist_sg_cnn")

In [3]:
import utils
import snn_utils
import base_model
import lenet_decolle_model

In [4]:
[torch.cuda.device(i) for i in range(torch.cuda.device_count())]

In [5]:
dir = "/media/user/EVIMO/raw/imo/eval/scene15_dyn_test_01/left_camera/ground_truth_000000"
sensor_size = [640, 480, 2]
batch_size = 4
framerate = 200
device = 'cuda'
epochs=40

In [6]:
# class EVIMO(Dataset):
#     def __init__(self,
#                  data: np.array,
#                  masks: list,
#                  transform: Optional[Callable] = None
#                 ):
#         #super().__init__(save_to, transform=transform)
#         self.masks = masks
#         if transform is not None:
#             self.data = transform(data)
#             if len(self.data) != len(self.masks):
#                 raise ValueError("Number of frames after transforms does not match number of masks")
#         else:
#             self.data = data

#         #target = [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
#         self.targets = []
        
#         num_masks_include = 0
#         for mask in self.masks:
#             if 23 in mask:
#                 num_masks_include += 1
#                 self.targets.append([0, 1])
#             else:
#                 self.targets.append([1, 0])

#         print(f"Target present in: {num_masks_include} of {len(self.masks)} masks. {num_masks_include/len(self.masks)}")
            

#         self.targets = np.asarray(self.targets)
        
#         #self.targets = torch.from_numpy(np.array(len(self.data)*[target]))
        

#     def __getitem__(self, index):
#         #return self.data[index], self.targets[index]
#         return {"data" : self.data[index], "mask": self.masks[index], "target": self.targets[index]}
#     def __len__(self) -> int: 
#         return len(self.data)
    

In [10]:
class EVIMO(Dataset):
    def __init__(self,
                 dir: str,
                 start_idx: int,
                 length: int,
                ):
        self.dir = dir
        self.length = length
        self.start_idx = start_idx

        self.labels = np.load(self.dir + "/labels.npy")

    def __getitem__(self, index):
        return {"data": np.load(self.dir + "/" + str(index + self.start_idx) + ".npy"), "target": self.labels[index + self.start_idx]}

    def __len__(self) -> int:
        return self.length - self.start_idx
    
    

In [11]:
# # Denoise removes isolated, one-off events
# # time_window
# frame_transform = transforms.Compose([#transforms.Denoise(filter_time=10000),
#                                       transforms.ToFrame(sensor_size=sensor_size,
#                                                          time_window=5000)
#                                      ])

In [12]:
tonic_dataset = EVIMO(dir="./data/EVIMO/train", start_idx=285, length=703)

In [13]:
1-0.2972972972972973

0.7027027027027026

In [14]:
def plot_frame(frame, mask): # TODO: Put the mask processing into bounding box into the tonic dataset?
    print(mask)
    img = frame[1] - frame[0]
    fig, ax = plt.subplots(1,2, figsize=(10,10))
    ax[0].imshow(img)

    mask = torch.from_numpy(mask)
    obj_ids = torch.unique(mask)
    #print(obj_ids)
    obj_ids = obj_ids[1:]
    print(obj_ids)
    detection_masks = mask == obj_ids[:, None, None]
    #print(detection_masks)
    
    boxes = masks_to_boxes(detection_masks)
    print(boxes)
    #for box in boxes:
    box = boxes[5]
    ax[0].add_patch(patches.Rectangle((box[0], box[1]), box[2]-box[0], box[3]-box[1], linewidth=1, edgecolor='r', facecolor='none'))


    ax[1].imshow(detection_masks[5])
    print(f"obj 23 in mask: {23 in mask}")
    
    ax[0].axis("off")
    ax[1].axis("off")
    plt.show()


#item = tonic_dataset.__getitem__(600)

#print(binned_events.shape)
#plot_frame(item['data'], item['mask'])

In [15]:
batched_trainloader = DataLoader(tonic_dataset, batch_size=batch_size, shuffle=True) # collate_fn=tonic.collation.PadTensors(),
single_trainloader = DataLoader(tonic_dataset, batch_size=1, shuffle=False)

In [16]:
#for i_batch, sample_batched in enumerate(trainloader):
#    print(i_batch, sample_batched['data'].size(), sample_batched['target'].size())

In [17]:
#datait = iter(batched_trainloader)
#batched_data = next(datait)

In [18]:
#batched_data['data']

In [19]:
#batched_data['target'].shape

In [20]:
item = next(iter(batched_trainloader))
data = item['data']
#target = item['target']

def decolle_loss(r, s, tgt):
    loss_tv = 0
    for i in range(len(r)):
        #print(f"Loss Readout shape : {r[i].shape}")
        #print(f"Loss Target shape : {tgt.shape}")
        loss_tv += loss(r[i],tgt) 
    return loss_tv

loss = torch.nn.SmoothL1Loss()

convnet_sg = lenet_decolle_model.LenetDECOLLE(out_channels=2,
                    Nhid=[32, 64, 128, 256], #Number of convolution channels
                    Mhid=[32, 64, 128],
                    kernel_size=[4, 8, 16, 32],
                    pool_size=[2, 4, 4, 8],
                    input_shape=[2, 480, 640],  # data.shape[1:],
                    alpha=[.95],
                    alpharp=[.65],
                    beta=[.92],
                    num_conv_layers=4,
                    num_mlp_layers=3,
                    lc_ampl=0.5).to(device)

#net = lenet_decolle_model.LenetDECOLLE(Nhid=[1,8],Mhid=[32,64],out_channels=2, input_shape=[2, 480, 640]).to(device)

#convnet_sg

data_d = data.to(device)
#target_d = target.to(device)
convnet_sg.init_parameters_evimo(data_d) # Modifies readout dim

LAYER SIZE: 2439712
STDV: 0.00032011109219189804
LAYER SIZE: 298304
STDV: 0.0009154623036203756
LAYER SIZE: 34048
STDV: 0.002709718654762875
LAYER SIZE: 512
STDV: 0.022097086912079608
LAYER SIZE: 32
STDV: 0.08838834764831843
LAYER SIZE: 64
STDV: 0.0625
LAYER SIZE: 128
STDV: 0.044194173824159216


In [21]:
#data_d.shape

In [22]:
#data_d.transpose(0, 1).shape

In [23]:
#data.shape[1:]

In [24]:
#target_d

In [25]:
#target_d.shape

In [26]:
opt_conv = torch.optim.Adamax(convnet_sg.get_trainable_parameters(), lr=1e-4, betas=[0., .95])

for e in tqdm(range(epochs), desc="Epoch"):        
    error = []
    accuracy=[]
    for item in iter(batched_trainloader):
        data = item['data']
        label = item['target']

        if label.shape[0] != 4:
            #print(f"last entry")
            break
        
        convnet_sg.train()
        loss_hist = 0
        data_d = data.to(device)
        label_d = label.to(device)
        convnet_sg.init_evimo(data_d)
        readout = 0
        
        st, rt, ut = convnet_sg.forward(data_d)        
        loss_tv = decolle_loss(rt, st, label_d)
        
        loss_tv.backward()
        opt_conv.step()
        opt_conv.zero_grad()
        loss_hist += loss_tv
        readout += rt[-1]

        #print(f"Readout: {readout}")
        error += (readout.argmax(axis=1)!=label_d.argmax(axis=1)).float()
        accuracy+=(readout.argmax(axis=1)==label_d.argmax(axis=1)).float()
        
    print('Training Error', torch.mean(torch.Tensor(error)).data)
    print('Training accuracy', torch.mean(torch.Tensor(accuracy)).data)     
    print('Epoch', e, 'Loss', loss_hist.data)
    PATH = './EVIMO_class_cnn.pth'  
    torch.save(convnet_sg.state_dict(), PATH)

Epoch:   0%|          | 0/40 [00:00<?, ?it/s]

Training Error tensor(0.5457)
Training accuracy tensor(0.4543)
Epoch 0 Loss tensor(1.4462, device='cuda:0')
Training Error tensor(0.4928)
Training accuracy tensor(0.5072)
Epoch 1 Loss tensor(1.2820, device='cuda:0')
Training Error tensor(0.4928)
Training accuracy tensor(0.5072)
Epoch 2 Loss tensor(1.4299, device='cuda:0')
Training Error tensor(0.4904)
Training accuracy tensor(0.5096)
Epoch 3 Loss tensor(1.3389, device='cuda:0')
Training Error tensor(0.5168)
Training accuracy tensor(0.4832)
Epoch 4 Loss tensor(1.3703, device='cuda:0')
Training Error tensor(0.4808)
Training accuracy tensor(0.5192)
Epoch 5 Loss tensor(1.3552, device='cuda:0')
Training Error tensor(0.4928)
Training accuracy tensor(0.5072)
Epoch 6 Loss tensor(1.2163, device='cuda:0')
Training Error tensor(0.4688)
Training accuracy tensor(0.5312)
Epoch 7 Loss tensor(1.3063, device='cuda:0')
Training Error tensor(0.5264)
Training accuracy tensor(0.4736)
Epoch 8 Loss tensor(1.3327, device='cuda:0')
Training Error tensor(0.5024

In [24]:
type(loss_tv)

torch.Tensor

In [27]:
error = []
accuracy=[]
y_pred = []
y_true = []
#convnet_sg.requires_init = True
for item in iter(batched_trainloader):
    #print("Item")
    #print(item)
    data = item['data']
    label = item['target']
    if label.shape[0] != 4:
        #print(f"last entry: {label}")
        break

    #print("earliest label")
    #print(label)
    #print(label.shape)

    
    loss_hist = 0
    data_d = data.to(device)
    label_d = label.to(device)
    print(f"data_d shape: {data_d.shape}")
    print(f"data_d_tranpose shape: {data_d.transpose(0,1).shape}")
    print(f"Len convnet: {len(convnet_sg)}")
    print(f"data init shape: {data_d.transpose(0,1)[:, 0, :, :].shape}")
    print(f"Test data shape: {data_d[0].shape}")
    
    convnet_sg.init_evimo(data_d)
    readout = 0
    with torch.no_grad():
        #for i in range(0, len(data_d)):
        #d = data_d[i:i+1]
        
        #print(d.shape)
        
        st, rt, ut = convnet_sg.forward(data_d)

        
        
        print(f"Label shape: {label_d.shape}")
        print("Readout")
        print(rt)
        print(rt[0].shape)

        #print("labels")
        #print(label_d)
        #print(label_d.shape)
        
        
        loss_tv = decolle_loss(rt, st, label_d)
        
        
        loss_hist += loss_tv
        readout += rt[-1]
        print(f"Readout: {readout}")
        output = (readout.argmax(axis=1)).data.cpu().numpy()
        print(f"Output: {output}")
        y_pred.extend(output)
        #labels = label_d.argmax(axis=1)
        print(f"Labels: {label_d}")
        labels = (label_d.argmax(axis=1)).data.cpu().numpy()
        print(f"Labels: {labels}")
        y_true.extend(labels)
        accuracy+=(readout.argmax(axis=1)==label_d.argmax(axis=1)).float()
        error += (readout.argmax(axis=1)!=label_d.argmax(axis=1)).float()
        break
        
print('Testing Error', torch.mean(torch.Tensor(error)).data)
print('Testing accuracy', torch.mean(torch.Tensor(accuracy)).data)

data_d shape: torch.Size([4, 2, 480, 640])
data_d_tranpose shape: torch.Size([2, 4, 480, 640])
Len convnet: 7
data init shape: torch.Size([2, 480, 640])
Test data shape: torch.Size([2, 480, 640])
Label shape: torch.Size([4, 2])
Readout
[tensor([[0.1892, 0.1854],
        [0.2004, 0.1661],
        [0.0088, 0.0892],
        [0.2712, 0.1167]], device='cuda:0'), tensor([[ 0.1112,  0.0547],
        [ 0.2483,  0.1166],
        [ 0.2613, -0.0366],
        [ 0.2711,  0.2221]], device='cuda:0'), tensor([[0.2608, 0.1217],
        [0.4984, 0.3228],
        [0.4693, 0.3460],
        [0.5119, 0.2287]], device='cuda:0'), tensor([[0.8808, 0.3838],
        [0.8262, 0.7793],
        [0.9797, 0.6277],
        [0.6476, 0.5369]], device='cuda:0'), tensor([[-0.2648,  0.4526],
        [-0.1730,  0.2042],
        [ 0.0883,  0.0401],
        [-0.2005,  0.1219]], device='cuda:0'), tensor([[0.4463, 0.0566],
        [0.1227, 0.1192],
        [0.2626, 0.0690],
        [0.2783, 0.5399]], device='cuda:0'), tensor([[

In [26]:
print(y_true)
print(y_pred)

[0, 1, 0, 0]
[0, 0, 0, 0]


In [27]:
"""
convnet_sg = lenet_decolle_model.LenetDECOLLE( out_channels=2,
                    Nhid=[16,32], #Number of convolution channels
                    Mhid=[64],
                    kernel_size=[7],
                    pool_size=[2,2],
                    input_shape=[2, 480, 640],  # data.shape[1:],
                    alpha=[.95],
                    alpharp=[.65],
                    beta=[.92],
                    num_conv_layers=2,
                    num_mlp_layers=1,
                    lc_ampl=.5).to(device)

"""

'\nconvnet_sg = lenet_decolle_model.LenetDECOLLE( out_channels=2,\n                    Nhid=[16,32], #Number of convolution channels\n                    Mhid=[64],\n                    kernel_size=[7],\n                    pool_size=[2,2],\n                    input_shape=[2, 480, 640],  # data.shape[1:],\n                    alpha=[.95],\n                    alpharp=[.65],\n                    beta=[.92],\n                    num_conv_layers=2,\n                    num_mlp_layers=1,\n                    lc_ampl=.5).to(device)\n\n'

In [28]:
net = lenet_decolle_model.LenetDECOLLE(Nhid=[16,32],Mhid=[64],out_channels=2, pool_size=[2,2], num_conv_layers=2, num_mlp_layers=1, lc_ampl=.5, input_shape=[2, 480, 640])
d = torch.zeros([1, 2, 480, 640])
print(d.shape)
st, rt, ut = net.forward(d)
print(rt[0].shape)

LAYER SIZE: 1228800
STDV: 0.00045105489780439506
LAYER SIZE: 614400
STDV: 0.000637887953849786
LAYER SIZE: 64
STDV: 0.0625
torch.Size([1, 2, 480, 640])
torch.Size([1, 2])


In [29]:
print(rt)

[tensor([[0.4393, 0.1550]], grad_fn=<AddmmBackward0>), tensor([[ 0.0854, -0.1504]], grad_fn=<AddmmBackward0>), tensor([[ 0.3901, -0.1543]], grad_fn=<AddmmBackward0>)]


In [30]:
target = torch.zeros([1, 2])
target

tensor([[0., 0.]])

In [31]:
loss_tv = decolle_loss(rt, st, target)